In [1]:
import numpy as np
import cv2
import tensorflow as tf
import time
import requests
import imutils

In [2]:
print(tf.__version__)

2.8.0


In [3]:
model = tf.keras.models.load_model("./GL_classification_model")
model.summary()

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 pooling_layer (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 dense_2 (Dense)             (None, 3)        

In [4]:
def smoothOutImgEdges(img):
    canny_img = cv2.Canny(img, 50, 140)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.dilate(canny_img , kernel, iterations = 4)
    return kernel, mask

In [5]:
def findBiggestContour(img):
    kernel, mask = smoothOutImgEdges(img)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    biggest_contour = 0
    biggest_area = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > biggest_area:
            biggest_area = area
            biggest_contour = np.array(contour).reshape((-1,1,2))
    return kernel, mask, biggest_contour

In [6]:
def findSmallContours(crop_mask):
    inverted_mask = cv2.bitwise_not(crop_mask)
    contours, _ = cv2.findContours(inverted_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    small_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 10000:
            small_contours.append(contour)
    return small_contours

In [7]:
def removeBackgroundNoise(crop_mask, kernel):
    crop_zero_mask = cv2.erode(crop_mask, kernel, iterations = 1)
    crop_zero_mask = cv2.dilate(crop_zero_mask, kernel, iterations = 1)
    crop_zero_mask = cv2.medianBlur(crop_zero_mask, 5)
    return crop_zero_mask

In [8]:
def masked_over_image(img):
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    kernel, mask, biggest_contour = findBiggestContour(gray_img)
    crop_zero_mask = np.zeros_like(mask)
    cv2.drawContours(crop_zero_mask, [biggest_contour], -1, (255), -1)
    
    small_contours = findSmallContours(crop_zero_mask)
    cv2.drawContours(crop_zero_mask, small_contours, -1, (255), -1)
    crop_zero_mask = removeBackgroundNoise(crop_zero_mask, kernel)
    crop = np.zeros_like(img)
    crop[crop_zero_mask == 255] = img[crop_zero_mask == 255]
    
    return crop

In [9]:
CLASSES = ['Compostable', 'Landfill', 'Recyclables']

In [10]:
def convert_and_predict_image(images, img_shape = 224):
    
    img = tf.convert_to_tensor(images)
    img = tf.image.resize(img, [img_shape, img_shape])
    img = tf.cast(img, tf.float32)

    return img

In [11]:
url = "http://192.168.184.197:8081/shot.jpg"

In [12]:
while (True):
    
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    frame = cv2.imdecode(img_arr, -1)
    frame = imutils.resize(frame, width=1000, height=1800)
    
    cv2.imshow("Normal", frame)
        
    crop_frame = masked_over_image(frame)
    crop_frame = cv2.cvtColor(crop_frame, cv2.COLOR_BGR2RGB)
        
    img = convert_and_predict_image(crop_frame) 
    pred_prob = model.predict(tf.expand_dims(img, axis=0)) #create a dimension 0 which is the batch size
    pred_class = CLASSES[pred_prob.argmax()] 
        
    crop_frame = cv2.putText(img = crop_frame, text = f"Predicted: {pred_class}", org = (20,30), 
                            fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                            color = (255,0,0), fontScale = 1, thickness = 2, lineType = cv2.LINE_AA)
        
    cv2.imshow("Prediction", crop_frame)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgproc\src\drawing.cpp:2509: error: (-215:Assertion failed) npoints > 0 in function 'cv::drawContours'
